# Random Forest Classification

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

## All CpG

In [2]:
%cd Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Constipation
df = pd.read_csv('ConstipationCateg_Methylome_APPRDXFilter.csv')
print(df.shape)
df.head(3)

Z:\PPMI_Data\Excels\NonMotors\Derived250\GeneCpG\Constipation
(458, 20757)


,Sentrix,PATNO,HYS,Gender,APPRDX,MDSP_Constipate,cg13609937,cg13847987,cg13822231,cg20728472,...,cg06819251,cg22158158,cg20161227,cg05701765,cg25437180,cg06002203,cg21722170,cg13142750,cg12743359,cg04293703
0,200991620021_R01C01,3001,2,1,1,0,0.020100,0.922355,0.011956,0.021950,...,0.900701,0.906393,0.948309,0.847871,0.647087,0.836282,0.731078,0.628030,0.883242,0.534427
1,200973410167_R04C01,3007,2,1,1,0,0.071529,0.938088,0.013730,0.029759,...,0.739334,0.888118,0.914366,0.854601,0.685887,0.890144,0.816051,0.735720,0.892971,0.540466
2,200973410167_R05C01,3010,2,1,1,0,0.068068,0.943100,0.013073,0.029993,...,0.795192,0.913136,0.914043,0.797516,0.623537,0.851161,0.849880,0.698082,0.863093,0.568347


In [3]:
df1 = df.drop(['Sentrix','PATNO','HYS','Gender','MDSP_Constipate'], axis=1)
df1.head(2)

,APPRDX,cg13609937,cg13847987,cg13822231,cg20728472,cg14567798,cg14205952,cg13083110,cg24453271,cg18015632,...,cg06819251,cg22158158,cg20161227,cg05701765,cg25437180,cg06002203,cg21722170,cg13142750,cg12743359,cg04293703
0,1,0.020100,0.922355,0.011956,0.021950,0.019629,0.011816,0.531190,0.824585,0.879564,...,0.900701,0.906393,0.948309,0.847871,0.647087,0.836282,0.731078,0.62803,0.883242,0.534427
1,1,0.071529,0.938088,0.013730,0.029759,0.025837,0.013719,0.578083,0.820133,0.892031,...,0.739334,0.888118,0.914366,0.854601,0.685887,0.890144,0.816051,0.73572,0.892971,0.540466


In [4]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.49000000000000005
2
Mean accuracy: 0.505
3
Mean accuracy: 0.5
4
Mean accuracy: 0.5399999999999999
5
Mean accuracy: 0.5250000000000001
6
Mean accuracy: 0.53
7
Mean accuracy: 0.475
8
Mean accuracy: 0.47000000000000003
9
Mean accuracy: 0.48500000000000004
10
Mean accuracy: 0.54
11
Mean accuracy: 0.495
12
Mean accuracy: 0.45500000000000007
13
Mean accuracy: 0.52
14
Mean accuracy: 0.43999999999999995
15
Mean accuracy: 0.4800000000000001
16
Mean accuracy: 0.545
17
Mean accuracy: 0.5
18
Mean accuracy: 0.5299999999999999
19
Mean accuracy: 0.545
20
Mean accuracy: 0.54
21
Mean accuracy: 0.5549999999999999
22
Mean accuracy: 0.5249999999999999
23
Mean accuracy: 0.5249999999999999
24
Mean accuracy: 0.5349999999999999
25
Mean accuracy: 0.5349999999999999
26
Mean accuracy: 0.475
27
Mean accuracy: 0.5650000000000001
28
Mean accuracy: 0.51
29
Mean accuracy: 0.505
30
Mean accuracy: 0.465
31
Mean accuracy: 0.575
32
Mean accuracy: 0.475
33
Mean accuracy: 0.48999999999999994
34
Mean accu

In [5]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Constipation_AllCpG_100RF.csv", index = False)

0.5065999999999999


## RanFor 1000 times. Samples = 50

In [6]:
df2 = df1.drop (['APPRDX'], axis=1)

In [7]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(50, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
        scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
        mean_score = scores.mean()
        print(f'Mean accuracy: {mean_score}')
        new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
        dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.5449999999999999
2
Mean accuracy: 0.51
3
Mean accuracy: 0.45
4
Mean accuracy: 0.5750000000000001
5
Mean accuracy: 0.52
6
Mean accuracy: 0.5549999999999999
7
Mean accuracy: 0.4600000000000001
8
Mean accuracy: 0.56
9
Mean accuracy: 0.48999999999999994
10
Mean accuracy: 0.525
11
Mean accuracy: 0.47000000000000003
12
Mean accuracy: 0.5599999999999999
13
Mean accuracy: 0.52
14
Mean accuracy: 0.545
15
Mean accuracy: 0.4600000000000001
16
Mean accuracy: 0.465
17
Mean accuracy: 0.5149999999999999
18
Mean accuracy: 0.55
19
Mean accuracy: 0.565
20
Mean accuracy: 0.485
21
Mean accuracy: 0.52
22
Mean accuracy: 0.505
23
Mean accuracy: 0.495
24
Mean accuracy: 0.5150000000000001
25
Mean accuracy: 0.5700000000000001
26
Mean accuracy: 0.555
27
Mean accuracy: 0.5499999999999999
28
Mean accuracy: 0.495
29
Mean accuracy: 0.45499999999999996
30
Mean accuracy: 0.44000000000000006
31
Mean accuracy: 0.49000000000000005
32
Mean accuracy: 0.4800000000000001
33
Mean accuracy: 0.495
34
Mean acc

In [8]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Constipation_Rand50CpG_100RF.csv", index = False)

0.5123999999999999


## RanFor 100 times. Samples = 93

In [9]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,101) :
        print(i)
        df3 = df2.sample(93, axis=1)
        df3 = df3.reindex(['APPRDX', *df3.columns], axis=1).assign(APPRDX=df1['APPRDX'])
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
        scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
        mean_score = scores.mean()
        print(f'Mean accuracy: {mean_score}')
        new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
        dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.53
2
Mean accuracy: 0.51
3
Mean accuracy: 0.445
4
Mean accuracy: 0.5599999999999999
5
Mean accuracy: 0.4600000000000001
6
Mean accuracy: 0.545
7
Mean accuracy: 0.51
8
Mean accuracy: 0.595
9
Mean accuracy: 0.44000000000000006
10
Mean accuracy: 0.51
11
Mean accuracy: 0.5349999999999999
12
Mean accuracy: 0.5599999999999999
13
Mean accuracy: 0.53
14
Mean accuracy: 0.5700000000000001
15
Mean accuracy: 0.49499999999999994
16
Mean accuracy: 0.5
17
Mean accuracy: 0.5149999999999999
18
Mean accuracy: 0.5
19
Mean accuracy: 0.535
20
Mean accuracy: 0.5250000000000001
21
Mean accuracy: 0.515
22
Mean accuracy: 0.51
23
Mean accuracy: 0.58
24
Mean accuracy: 0.445
25
Mean accuracy: 0.5900000000000001
26
Mean accuracy: 0.535
27
Mean accuracy: 0.56
28
Mean accuracy: 0.5900000000000001
29
Mean accuracy: 0.48
30
Mean accuracy: 0.48
31
Mean accuracy: 0.45499999999999996
32
Mean accuracy: 0.5700000000000001
33
Mean accuracy: 0.51
34
Mean accuracy: 0.485
35
Mean accuracy: 0.505
36
Mean accu

In [10]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Constipation_Rand93CpG_100RF.csv", index = False)

0.5147000000000002


## RanFor Top 50 Boruta 100 times. Samples = 50

In [11]:
df = pd.read_csv('Constipation_Boruta_Top50_Data.csv')
print(df.shape)
df.head(2)

(458, 52)


,PATNO,APPRDX,cg22237213,cg13332222,cg14005240,cg14889643,cg17612987,cg21043558,cg12298429,cg27281389,...,cg03478448,cg20187858,cg09233704,cg05142211,cg12683454,cg09043524,cg14196331,cg26458329,cg19371224,cg02454595
0,3001,1,0.922623,0.902238,0.878486,0.017995,0.031849,0.667449,0.032801,0.026921,...,0.636575,0.164275,0.620103,0.035018,0.023826,0.926060,0.034453,0.029374,0.751572,0.058504
1,3007,1,0.918179,0.928699,0.887782,0.020710,0.031061,0.659961,0.042715,0.033420,...,0.605065,0.122112,0.682452,0.041906,0.028970,0.898801,0.028081,0.021559,0.855625,0.042324


In [12]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg22237213,cg13332222,cg14005240,cg14889643,cg17612987,cg21043558,cg12298429,cg27281389,cg25434660,...,cg03478448,cg20187858,cg09233704,cg05142211,cg12683454,cg09043524,cg14196331,cg26458329,cg19371224,cg02454595
0,1,0.922623,0.902238,0.878486,0.017995,0.031849,0.667449,0.032801,0.026921,0.848560,...,0.636575,0.164275,0.620103,0.035018,0.023826,0.926060,0.034453,0.029374,0.751572,0.058504
1,1,0.918179,0.928699,0.887782,0.020710,0.031061,0.659961,0.042715,0.033420,0.864457,...,0.605065,0.122112,0.682452,0.041906,0.028970,0.898801,0.028081,0.021559,0.855625,0.042324


In [13]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.6199999999999999
2
Mean accuracy: 0.635
3
Mean accuracy: 0.6449999999999999
4
Mean accuracy: 0.65
5
Mean accuracy: 0.6599999999999999
6
Mean accuracy: 0.66
7
Mean accuracy: 0.635
8
Mean accuracy: 0.635
9
Mean accuracy: 0.625
10
Mean accuracy: 0.65
11
Mean accuracy: 0.5650000000000001
12
Mean accuracy: 0.6199999999999999
13
Mean accuracy: 0.64
14
Mean accuracy: 0.655
15
Mean accuracy: 0.6
16
Mean accuracy: 0.63
17
Mean accuracy: 0.615
18
Mean accuracy: 0.7000000000000001
19
Mean accuracy: 0.63
20
Mean accuracy: 0.6849999999999999
21
Mean accuracy: 0.605
22
Mean accuracy: 0.6050000000000001
23
Mean accuracy: 0.67
24
Mean accuracy: 0.6149999999999999
25
Mean accuracy: 0.6000000000000001
26
Mean accuracy: 0.54
27
Mean accuracy: 0.695
28
Mean accuracy: 0.6049999999999999
29
Mean accuracy: 0.6449999999999999
30
Mean accuracy: 0.6300000000000001
31
Mean accuracy: 0.6950000000000001
32
Mean accuracy: 0.595
33
Mean accuracy: 0.6199999999999999
34
Mean accuracy: 0.635
35
Mean 

In [14]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Constipation_Boruta50CpG_100RF.csv", index = False)

0.6372


## RanFor Top 50 NMI 100 times. Samples = 50

In [15]:
df = pd.read_csv('Constipation_NMI_Top50_Data.csv')
print(df.shape)
df.head(2)

(458, 51)


,APPRDX,cg13605579,cg25410117,cg10688123,cg13332222,cg23786063,cg20349377,cg25210047,cg10044815,cg12298429,...,cg12036621,cg26801812,cg09068129,cg06425631,cg05142211,cg11549065,cg17458425,cg10783204,cg08096908,cg24224407
0,1,0.863394,0.133229,0.833705,0.902238,0.889644,0.876278,0.827441,0.894322,0.032801,...,0.030171,0.847042,0.852115,0.932296,0.035018,0.032903,0.856462,0.585492,0.905542,0.875350
1,1,0.893290,0.090223,0.861144,0.928699,0.893976,0.893810,0.804296,0.902732,0.042715,...,0.033681,0.817918,0.819744,0.930258,0.041906,0.043359,0.908599,0.654016,0.918975,0.867527


In [16]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.61
2
Mean accuracy: 0.635
3
Mean accuracy: 0.6950000000000001
4
Mean accuracy: 0.635
5
Mean accuracy: 0.655
6
Mean accuracy: 0.6500000000000001
7
Mean accuracy: 0.685
8
Mean accuracy: 0.6500000000000001
9
Mean accuracy: 0.67
10
Mean accuracy: 0.65
11
Mean accuracy: 0.6050000000000001
12
Mean accuracy: 0.6649999999999999
13
Mean accuracy: 0.69
14
Mean accuracy: 0.6849999999999999
15
Mean accuracy: 0.67
16
Mean accuracy: 0.675
17
Mean accuracy: 0.65
18
Mean accuracy: 0.6900000000000001
19
Mean accuracy: 0.675
20
Mean accuracy: 0.675
21
Mean accuracy: 0.635
22
Mean accuracy: 0.67
23
Mean accuracy: 0.665
24
Mean accuracy: 0.595
25
Mean accuracy: 0.675
26
Mean accuracy: 0.6799999999999999
27
Mean accuracy: 0.6900000000000001
28
Mean accuracy: 0.625
29
Mean accuracy: 0.71
30
Mean accuracy: 0.585
31
Mean accuracy: 0.6950000000000001
32
Mean accuracy: 0.645
33
Mean accuracy: 0.6699999999999999
34
Mean accuracy: 0.6699999999999999
35
Mean accuracy: 0.7100000000000001
36
Mean 

In [17]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Constipation_NMI50CpG_100RF.csv", index = False)

0.6580000000000001


## RanFor Top 50 Boruta + NMI 100 times. Samples = 93

In [18]:
df = pd.read_csv('Constipation_Top50sMerge_Data.csv')
print(df.shape)
df.head(2)

(458, 95)


,PATNO,APPRDX,cg22237213,cg13332222,cg14005240,cg14889643,cg17612987,cg21043558,cg12298429,cg27281389,...,cg07087293,cg12036621,cg26801812,cg09068129,cg06425631,cg11549065,cg17458425,cg10783204,cg08096908,cg24224407
0,3001,1,0.922623,0.902238,0.878486,0.017995,0.031849,0.667449,0.032801,0.026921,...,0.074554,0.030171,0.847042,0.852115,0.932296,0.032903,0.856462,0.585492,0.905542,0.875350
1,3007,1,0.918179,0.928699,0.887782,0.020710,0.031061,0.659961,0.042715,0.033420,...,0.067149,0.033681,0.817918,0.819744,0.930258,0.043359,0.908599,0.654016,0.918975,0.867527


In [19]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg22237213,cg13332222,cg14005240,cg14889643,cg17612987,cg21043558,cg12298429,cg27281389,cg25434660,...,cg07087293,cg12036621,cg26801812,cg09068129,cg06425631,cg11549065,cg17458425,cg10783204,cg08096908,cg24224407
0,1,0.922623,0.902238,0.878486,0.017995,0.031849,0.667449,0.032801,0.026921,0.848560,...,0.074554,0.030171,0.847042,0.852115,0.932296,0.032903,0.856462,0.585492,0.905542,0.875350
1,1,0.918179,0.928699,0.887782,0.020710,0.031061,0.659961,0.042715,0.033420,0.864457,...,0.067149,0.033681,0.817918,0.819744,0.930258,0.043359,0.908599,0.654016,0.918975,0.867527


In [20]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])

for i in range(1, 101):
    print(i)
    df2 = df1.groupby('APPRDX').apply(lambda x: x.sample(100))
    X = df2.drop('APPRDX', axis = 1)
    y = df2['APPRDX']
    random_forest_model = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'gini')
    scores = cross_val_score(random_forest_model, X, y, cv = 10, n_jobs = -1)
    mean_score = scores.mean()
    print(f'Mean accuracy: {mean_score}')
    new_row = {'Accuracy': mean_score, 'Std_Dev': std(scores)}
    dframe1 = pd.concat([dframe1, pd.DataFrame(new_row, index=[0])], ignore_index=True)


1
Mean accuracy: 0.6599999999999999
2
Mean accuracy: 0.6000000000000001
3
Mean accuracy: 0.635
4
Mean accuracy: 0.655
5
Mean accuracy: 0.6799999999999999
6
Mean accuracy: 0.6849999999999999
7
Mean accuracy: 0.6649999999999999
8
Mean accuracy: 0.6
9
Mean accuracy: 0.665
10
Mean accuracy: 0.6199999999999999
11
Mean accuracy: 0.635
12
Mean accuracy: 0.635
13
Mean accuracy: 0.65
14
Mean accuracy: 0.715
15
Mean accuracy: 0.6199999999999999
16
Mean accuracy: 0.665
17
Mean accuracy: 0.6449999999999999
18
Mean accuracy: 0.7100000000000001
19
Mean accuracy: 0.665
20
Mean accuracy: 0.6799999999999999
21
Mean accuracy: 0.63
22
Mean accuracy: 0.6500000000000001
23
Mean accuracy: 0.6649999999999999
24
Mean accuracy: 0.5599999999999999
25
Mean accuracy: 0.6849999999999998
26
Mean accuracy: 0.635
27
Mean accuracy: 0.705
28
Mean accuracy: 0.615
29
Mean accuracy: 0.67
30
Mean accuracy: 0.675
31
Mean accuracy: 0.6950000000000001
32
Mean accuracy: 0.61
33
Mean accuracy: 0.63
34
Mean accuracy: 0.684999999

In [21]:
print(mean(dframe1['Accuracy']))
dframe1.to_csv ("Constipation_Top50sCpG93_100RF.csv", index = False)

0.6545999999999998
